In [66]:
import os
import shutil
import json
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [55]:
DATA_DIR = '../datasets/fsoco/fsoco_bounding_boxes_train/'
EXPORT_DIR = '../datasets/fsoco/yolo_cones/'

Class Reference:
- 0: Large Orange Cone
- 1: Blue Cone
- 2: Yellow Cne

In [65]:
# Convert json to yolo coords
for u in os.listdir(DATA_DIR):
    if u.split('.')[-1] != 'json':
        print(u)
        UNI_DIR = f'{DATA_DIR}{u}/'
        for i in os.listdir(f'{UNI_DIR}img'):
            name = i.split('.')[0]
            image = Image.open(f'{UNI_DIR}img/{i}')
            with open(f'{UNI_DIR}ann/{i}.json', 'r') as meta:
                meta = json.loads(meta.read())

            # fig, ax = plt.subplots()
            # ax.imshow(image)
            img_width, img_height = image.size
            print(name)
            print('Cones Found:', len(meta['objects']))

            yolo_labels = ''
            
            for cone in meta['objects']:
                # Get Class
                cone_class = cone['classTitle']
                w_left = cone['points']['exterior'][0][0]
                h_top = cone['points']['exterior'][0][1]
                w_diff = cone['points']['exterior'][1][0]
                h_diff = cone['points']['exterior'][1][1]
                height = h_diff-h_top
                width = w_diff-w_left
                if 'blue' in cone_class:
                    color = 'blue'
                    cone_integer = 1
                elif 'yellow' in cone_class:
                    color = 'yellow'
                    cone_integer = 2
                else:
                    color = 'orange'
                    cone_integer = 0
                #ax.add_patch(Rectangle((w_left, h_top), width, height, edgecolor = color, facecolor='none'))

                # Get Yolo Conversion
                yolo_width = width/img_width
                yolo_height = height/img_height
                yolo_left_center = (w_left+(width/2))/img_width
                yolo_top_center = (h_top+(height/2))/img_height

                yolo_labels += f'{cone_integer} {yolo_left_center} {yolo_top_center} {yolo_width} {yolo_height}\n'

                # Save Image

                #image.save(f'{EXPORT_DIR}images/all/{i}')
                shutil.copy(f'{UNI_DIR}img/{i}', f'{EXPORT_DIR}images/all/{i}')

                # Save labels
                with open(f'{EXPORT_DIR}labels/all/{name}.txt', 'w') as f:
                    f.write(yolo_labels)



ugatu
amz_03547
Cones Found: 38
amz_02251
Cones Found: 36
amz_01920
Cones Found: 12
amz_03613
Cones Found: 8
amz_03257
Cones Found: 26
amz_00992
Cones Found: 21
amz_01336
Cones Found: 11
amz_03228
Cones Found: 3
amz_04466
Cones Found: 14
amz_03996
Cones Found: 42
amz_01102
Cones Found: 1
amz_02844
Cones Found: 87
amz_01905
Cones Found: 17
amz_01545
Cones Found: 102
amz_02179
Cones Found: 47
amz_00816
Cones Found: 16
amz_01695
Cones Found: 22
amz_04469
Cones Found: 14
amz_02512
Cones Found: 7
amz_03460
Cones Found: 54
amz_01847
Cones Found: 11
amz_04261
Cones Found: 16
amz_01326
Cones Found: 41
amz_04404
Cones Found: 23
amz_03501
Cones Found: 32
amz_04272
Cones Found: 7
amz_03764
Cones Found: 4
amz_02754
Cones Found: 3
amz_04240
Cones Found: 8
amz_02434
Cones Found: 8
amz_00982
Cones Found: 75
amz_03659
Cones Found: 31
amz_04376
Cones Found: 4
amz_02444
Cones Found: 10
amz_02773
Cones Found: 2
amz_04022
Cones Found: 12
amz_04125
Cones Found: 4
amz_04342
Cones Found: 27
amz_02912
Cones F

### Create Train/Val/Test Split
70/15/15 ratio of random data

In [ ]:
random.seed(42)
all_samples = os.listdir(f'{EXPORT_DIR}images/all')
random.shuffle(all_samples)

['BME_01157.jpg', 'ka_00890.png', 'prom_00202.jpg', 'BME_01044.jpg', 'amz_03290.png', 'amz_01752.png', 'ff_00544.png', 'FRT_00175.png', 'ulm_00389.jpg', 'ka_01196.png']


In [78]:
train_size = round(len(all_samples)*0.7)
val_size = round(len(all_samples)*0.15)

In [79]:
train = all_samples[:train_size]
val = all_samples[train_size:train_size+val_size]
test = all_samples[train_size+val_size:]

In [87]:
# Copy files into right export dir
for i in train:
    label_name = i.split('.')[0]
    shutil.copy(f'{EXPORT_DIR}images/all/{i}', f'{EXPORT_DIR}images/train/{i}')
    shutil.copy(f'{EXPORT_DIR}labels/all/{label_name}.txt', f'{EXPORT_DIR}labels/train/{label_name}.txt')

for i in val:
    label_name = i.split('.')[0]
    shutil.copy(f'{EXPORT_DIR}images/all/{i}', f'{EXPORT_DIR}images/val/{i}')
    shutil.copy(f'{EXPORT_DIR}labels/all/{label_name}.txt', f'{EXPORT_DIR}labels/val/{label_name}.txt')

for i in test:
    label_name = i.split('.')[0]
    shutil.copy(f'{EXPORT_DIR}images/all/{i}', f'{EXPORT_DIR}images/test/{i}')
    shutil.copy(f'{EXPORT_DIR}labels/all/{label_name}.txt', f'{EXPORT_DIR}labels/test/{label_name}.txt')